## MOVIES

In [11]:
import pandas as pd
import selenium.webdriver.chrome.webdriver
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import random
import numpy as np
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import re

# Set up the WebDriver
options = Options()
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=options)
driver.get("https://www.imdb.com/chart/top/")

# Collect data
ratings = [elem.text for elem in driver.find_elements(By.CLASS_NAME, 'ipc-rating-star--rating')]
rating_by = [elem.text for elem in driver.find_elements(By.CLASS_NAME, 'ipc-rating-star--voteCount')]

years = []
hours = []

for i in range(1, 251):
    # Extract Year
    year_xpath = f'//*[@id="__next"]/main/div/div[3]/section/div/div[2]/div/ul/li[{i}]/div[2]/div/div/div[2]/span[1]'
    year_elements = driver.find_elements(By.XPATH, year_xpath)
    years.extend([elem.text for elem in year_elements])

    # Extract Hours
    hours_xpath = f'//*[@id="__next"]/main/div/div[3]/section/div/div[2]/div/ul/li[{i}]/div[2]/div/div/div[2]/span[2]'
    hours_elements = driver.find_elements(By.XPATH, hours_xpath)
    hours.extend([elem.text for elem in hours_elements])

# Ensure all lists have the same length
min_length = min(len(ratings), len(rating_by), len(years), len(hours))
data = {
    'Rating': ratings[:min_length],
    'RateBy': rating_by[:min_length],
    'Year': years[:min_length],
    'Hours': hours[:min_length]
}

# Create a DataFrame and save to Excel
df_desc = pd.DataFrame(data)

# Using inspect by class, it returns also the title of the page, which I don't want it, I want it to start from the movie title only. so excluding this first from the process, and call it later.
titles = [elem.text for elem in driver.find_elements(By.CLASS_NAME, 'ipc-title__text')[2:252]]
df_title = pd.DataFrame(titles).rename(columns={0: 'Title'})
dfMerge1 = pd.merge(df_title, df_desc, left_index=True, right_index=True, how='left')

# Remove leading indices from 'Title'
dfMerge1['Title'] = dfMerge1['Title'].str.replace(r'^\d+\.\s*', '', regex=True)

# Turn the Value to numeric so easier to count later
def convert_value(value):
    """
    Convert string values with 'M' (millions) or 'K' (thousands) to integers.
    """
    value = value.strip().strip('()')
    if 'M' in value:
        return int(float(value.replace('M', '')) * 1_000_000)
    elif 'K' in value:
        return int(float(value.replace('K', '')) * 1_000)
    else:
        return int(value)

# Apply conversion to 'RatingBy' and cast to integer
dfMerge1['RateBy'] = dfMerge1['RateBy'].apply(convert_value).astype(int)

# Convert 'Rating' and 'Year' to float
dfMerge1['Rating'] = dfMerge1['Rating'].astype(float)
dfMerge1['Year'] = dfMerge1['Year'].astype(int)

def convert_to_minutes(duration):
    """
    Convert a duration string to total minutes.
    """
    hours = re.search(r'(\d+)h', duration)
    minutes = re.search(r'(\d+)m', duration)
    
    hours = int(hours.group(1)) if hours else 0
    minutes = int(minutes.group(1)) if minutes else 0
    
    return hours * 60 + minutes

# Apply duration conversion to 'Hours' column
dfMerge1['Minutes'] = dfMerge1['Hours'].apply(convert_to_minutes)
dfMerge1 = dfMerge1.drop(columns='Hours')
dfMerge1 = dfMerge1.assign(
    Type='Movies',
    Rank=range(1, len(dfMerge1) + 1),
    Segment='Top 250 Movies'
)

dfMerge1

,Title,Rating,RateBy,Year,Minutes,Type,Rank,Segment
0,The Shawshank Redemption,9.3,2900000,1994,142,Movies,1,Top 250 Movies
1,The Godfather,9.2,2000000,1972,175,Movies,2,Top 250 Movies
2,The Dark Knight,9.0,2900000,2008,152,Movies,3,Top 250 Movies
3,The Godfather Part II,9.0,1400000,1974,202,Movies,4,Top 250 Movies
4,12 Angry Men,9.0,877000,1957,96,Movies,5,Top 250 Movies
...,...,...,...,...,...,...,...,...
245,Cool Hand Luke,8.1,190000,1967,127,Movies,246,Top 250 Movies
246,The Help,8.1,497000,2011,146,Movies,247,Top 250 Movies
247,It Happened One Night,8.1,113000,1934,105,Movies,248,Top 250 Movies
248,Aladdin,8.0,473000,1992,90,Movies,249,Top 250 Movies


## TV SHOWS

In [13]:
options = Options()
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=options)
driver.get("https://m.imdb.com/chart/toptv/")

ratings = [elem.text for elem in driver.find_elements(By.CLASS_NAME, 'ipc-rating-star--rating')]
rating_by = [elem.text for elem in driver.find_elements(By.CLASS_NAME, 'ipc-rating-star--voteCount')]

years = []
episode = []

for i in range(1, 251):
    # Extract Year
    year_xpath = f'//*[@id="__next"]/main/div/div[3]/section/div/div[2]/div/ul/li[{i}]/div[2]/div/div/div[2]/span[1]'
    year_elements = driver.find_elements(By.XPATH, year_xpath)
    years.extend([elem.text for elem in year_elements])

    # Extract Hours
    hours_xpath = f'//*[@id="__next"]/main/div/div[3]/section/div/div[2]/div/ul/li[{i}]/div[2]/div/div/div[2]/span[2]'
    hours_elements = driver.find_elements(By.XPATH, hours_xpath)
    episode.extend([elem.text for elem in hours_elements])

# Ensure all lists have the same length
min_length = min(len(ratings), len(rating_by), len(years), len(episode))
data = {
    'Rating': ratings[:min_length],
    'RateBy': rating_by[:min_length],
    'Year': years[:min_length],
    'Episode': episode[:min_length]
}

# Create a DataFrame and save to Excel
df_desc = pd.DataFrame(data)

# Using inspect by class, it returns also the title of the page, which I don't want it, I want it to start from the movie title only. so excluding this first from the process, and call it later.
titles = [elem.text for elem in driver.find_elements(By.CLASS_NAME, 'ipc-title__text')[2:252]]
df_title = pd.DataFrame(titles).rename(columns={0: 'Title'})
dfMerge2 = pd.merge(df_title, df_desc, left_index=True, right_index=True, how='left')


# Remove leading indices from 'Title'
dfMerge2['Title'] = dfMerge2['Title'].str.replace(r'^\d+\.\s*', '', regex=True)

# Turn the Value to numeric so easier to count later
def convert_value(value):
    """
    Convert string values with 'M' (millions) or 'K' (thousands) to integers.
    """
    value = value.strip().strip('()')
    if 'M' in value:
        return int(float(value.replace('M', '')) * 1_000_000)
    elif 'K' in value:
        return int(float(value.replace('K', '')) * 1_000)
    else:
        return int(value)

# Apply conversion to 'RatingBy' and cast to integer
dfMerge2['RateBy'] = dfMerge2['RateBy'].apply(convert_value).astype(int)

# Convert 'Rating' and 'Year' to float
dfMerge2['Rating'] = dfMerge2['Rating'].astype(float)

def extract_first_year(year_string):
    """
    Extract the first year from a string that may contain a range of years.
    """
    # Split by '–' or '-' if it's a range
    years = re.split(r'–|-', year_string)
    # Return the first year
    return years[0].strip()

# Apply the function to the 'Year' column
dfMerge2['Year'] = dfMerge2['Year'].apply(extract_first_year)
dfMerge2['Episode'] = dfMerge2['Episode'].str.replace(r'\s*eps', '', regex=True).astype(int)
dfMerge2 = dfMerge2.assign(
    Type='TV Shows',
    Rank=range(1, len(dfMerge2) + 1),
    Segment='Top 250 TV Shows'
)
dfMerge2

,Title,Rating,RateBy,Year,Episode,Type,Rank,Segment
0,Breaking Bad,9.5,2200000,2008,62,TV Shows,1,Top 250 TV Shows
1,Planet Earth II,9.5,160000,2016,6,TV Shows,2,Top 250 TV Shows
2,Planet Earth,9.4,222000,2006,11,TV Shows,3,Top 250 TV Shows
3,Band of Brothers,9.4,536000,2001,10,TV Shows,4,Top 250 TV Shows
4,Chernobyl,9.3,883000,2019,5,TV Shows,5,Top 250 TV Shows
...,...,...,...,...,...,...,...,...
245,Community,8.5,296000,2009,110,TV Shows,246,Top 250 TV Shows
246,Sapne Vs Everyone,9.4,69000,2023,5,TV Shows,247,Top 250 TV Shows
247,Tear Along the Dotted Line,8.6,15000,2021,6,TV Shows,248,Top 250 TV Shows
248,Gintama,8.7,16000,2005,375,TV Shows,249,Top 250 TV Shows


## COMBINE TOP 250

In [17]:
df = pd.concat([dfMerge1, dfMerge2], ignore_index=True)
df = df.assign(
    Episode=lambda df: df['Episode'].fillna(1).astype(int),
    Minutes=lambda df: df['Minutes'].fillna(0).astype(int)
)

df

,Title,Rating,RateBy,Year,Minutes,Type,Rank,Segment,Episode
0,The Shawshank Redemption,9.3,2900000,1994,142,Movies,1,Top 250 Movies,1
1,The Godfather,9.2,2000000,1972,175,Movies,2,Top 250 Movies,1
2,The Dark Knight,9.0,2900000,2008,152,Movies,3,Top 250 Movies,1
3,The Godfather Part II,9.0,1400000,1974,202,Movies,4,Top 250 Movies,1
4,12 Angry Men,9.0,877000,1957,96,Movies,5,Top 250 Movies,1
...,...,...,...,...,...,...,...,...,...
495,Community,8.5,296000,2009,0,TV Shows,246,Top 250 TV Shows,110
496,Sapne Vs Everyone,9.4,69000,2023,0,TV Shows,247,Top 250 TV Shows,5
497,Tear Along the Dotted Line,8.6,15000,2021,0,TV Shows,248,Top 250 TV Shows,6
498,Gintama,8.7,16000,2005,0,TV Shows,249,Top 250 TV Shows,375


In [23]:
df.to_excel('IMDBTop250.xlsx')